In [1]:
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import InceptionV3
from keras.applications.vgg16 import preprocess_input

In [2]:
num_classes = 4
image_size = 224
batch_size_training = 10
batch_size_validation = 10


In [3]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input,)

In [4]:
train_generator = data_generator.flow_from_directory(
    'New folder/train(different images)',
    target_size = (image_size, image_size),
    batch_size = batch_size_training,
    class_mode = 'categorical'
    )

validation_generator = data_generator.flow_from_directory(
    'New folder/validation',
    target_size = (image_size, image_size),
    batch_size = batch_size_training,
    class_mode = 'categorical'
    )

Found 315 images belonging to 4 classes.
Found 9 images belonging to 4 classes.


In [5]:
#adding VGG16 model
from xml.etree.ElementInclude import include


base_model = InceptionV3(include_top = False, pooling='avg',input_shape=[image_size, image_size, 3], weights ='imagenet')
base_model.trainable = False

In [6]:
from tensorflow.keras import layers, models

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(4096, activation='relu')
dense_layer_2 = layers.Dense(4096, activation='relu')
prediction_layer = layers.Dense(4, activation='softmax')


model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 2048)              21802784  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 dense (Dense)               (None, 4096)              8392704   
                                                                 
 dense_1 (Dense)             (None, 4096)              16781312  
                                                                 
 dense_2 (Dense)             (None, 4)                 16388     
                                                                 
Total params: 46,993,188
Trainable params: 25,190,404
Non-trainable params: 21,802,784
_________________________________________________________________


In [8]:
from tensorflow.keras.callbacks import EarlyStopping

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'],
)

steps_per_epoch_train = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
number_epochs = 5




#model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32, callbacks=[es])

In [9]:
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5,  restore_best_weights=True)

#early stopping is not used
fit_history = model.fit(train_generator, epochs=number_epochs,verbose=1, validation_data=validation_generator, callbacks=[es])

Epoch 1/5
23/32 [====================>.........] - ETA: 3s - loss: 66.3826 - accuracy: 0.2933

c:\Users\ASUS\anaconda3\lib\site-packages\PIL\Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


32/32 [==============================] - 20s 482ms/step - loss: 47.9865 - accuracy: 0.3365 - val_loss: 1.2444 - val_accuracy: 0.5556
Epoch 2/5
32/32 [==============================] - 14s 439ms/step - loss: 0.7276 - accuracy: 0.7143 - val_loss: 0.8040 - val_accuracy: 0.7778
Epoch 3/5
32/32 [==============================] - 14s 426ms/step - loss: 0.9224 - accuracy: 0.6889 - val_loss: 0.8041 - val_accuracy: 0.6667
Epoch 4/5
32/32 [==============================] - 14s 433ms/step - loss: 0.4606 - accuracy: 0.8159 - val_loss: 0.7701 - val_accuracy: 0.6667
Epoch 5/5
32/32 [==============================] - 14s 427ms/step - loss: 0.5710 - accuracy: 0.7714 - val_loss: 0.4559 - val_accuracy: 0.7778


In [10]:
model.save('model/Inception/Inception_Product_Classification_3Dense_different_imgs.h5')

In [1]:
from keras.models import load_model
vgg16_saved = load_model('model/Inception/Inception_Product_Classification_3Dense_different_imgs.h5')

In [2]:
from keras.models import Model
basemodel = Model(inputs=vgg16_saved.input, outputs=vgg16_saved.get_layer('dense').output)

In [3]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image2 = load_img('New folder/test/test3.png', target_size=(224, 224))
image1 = load_img('New folder/test/test4.jpg', target_size=(224, 224))

from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel.predict(image1)
yhat2_base = basemodel.predict(image2)

# import required libraries
import numpy as np
from numpy.linalg import norm

A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

#0.94 when same image is used to train the model

1/1 [==============================] - 0s 143ms/step
Cosine Similarity: 0.87195355


In [4]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test.png', target_size=(224, 224))
image2 = load_img('New folder/test/test3.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel.predict(image1)
yhat2_base = basemodel.predict(image2)

A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)
#0.9287 - when same image is used to train the model

1/1 [==============================] - 0s 100ms/step
Cosine Similarity: 0.90706974


In [5]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test.png', target_size=(224, 224))
image2 = load_img('New folder/test/test4.jpg', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel.predict(image1)
yhat2_base = basemodel.predict(image2)

A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)
#0.81 - when same image is used to train the model 

1/1 [==============================] - 0s 98ms/step
Cosine Similarity: 0.6447912


In [6]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test2.jpg', target_size=(224, 224))
image2 = load_img('New folder/test/test3.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel.predict(image1)
yhat2_base = basemodel.predict(image2)

A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

#0.6523 when same image is used to train the model

1/1 [==============================] - 0s 101ms/step
Cosine Similarity: 0.56185156


In [7]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test2.jpg', target_size=(224, 224))
image2 = load_img('New folder/test/test5.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel.predict(image1)
yhat2_base = basemodel.predict(image2)

A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

1/1 [==============================] - 0s 79ms/step
Cosine Similarity: 0.47186774


In [10]:
#predicting
from tensorflow.keras.preprocessing.image import load_img
# load an image from file
image1 = load_img('New folder/test/test3.png', target_size=(224, 224))
image2 = load_img('New folder/test/test5.png', target_size=(224, 224))


from tensorflow.keras.preprocessing.image import img_to_array
image1 = img_to_array(image1)
image2 = img_to_array(image2)

# reshape data for the model
image1 = image1.reshape((1, image1.shape[0], image1.shape[1], image1.shape[2]))
image2 = image2.reshape((1, image2.shape[0], image2.shape[1], image2.shape[2]))


from keras.applications.vgg16 import preprocess_input
# prepare the image for the VGG model
image1 = preprocess_input(image1)
image2 = preprocess_input(image2)

# predict the probability across all output classes
yhat1_base = basemodel.predict(image1)
yhat2_base = basemodel.predict(image2)

A = yhat1_base[0]
B = yhat2_base[0]

cosine = np.dot(A,B)/(norm(A)*norm(B))
print("Cosine Similarity:", cosine)

1/1 [==============================] - 0s 96ms/step
Cosine Similarity: 0.7437713
